In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State, ALL
import numpy as np
import plotly.graph_objects as go
from scipy.spatial import ConvexHull

import pandas as pd
import matplotlib.pyplot as plt
import json

from helpers import fill_template, read_config, read_json
from inference import KnnClassifier

user_x, user_y = [0.5, 0, 0], 2


config = read_config()
model = KnnClassifier( visualization_method = "mde", config = config )
model.embedder.get_embeddings ## Compute the roles embeddings if not already in embeddings.txt file.
df = model.embedder.get_initial_corrdinates()
X = df[["x","y","z"]].values
labels = df["role_id"].values
representatives = np.unique(labels)
rol_to_id_dict = read_json("./data/role_to_id.json")



/Users/rayenebech/Desktop/rayene/phd_application/Helsinki/studies/Period_1/Introduction_to_data_science/proje-env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


Dimensionality Reduction with mde took: 21.094807863235474


In [2]:
with open('./data/suggestions.json', 'r') as f:
    form_dict = json.load(f)

## create a Div for each dropdown from the key and list of values from 
def group_from_json(key, values):
    return html.Div([
                html.Label(f"{key}:"),
                dcc.Dropdown(id={'type': 'input-field', 'key': key}, 
                             options=[{'label': option, 'value': option} for option in values], multi=True,),
                    ], className='form-group')

divs = [group_from_json(key, values) for key, values in form_dict.items()]
form = html.Div(divs, className='form-container')

In [3]:

# Create the Dash app
app = dash.Dash(__name__)

# Target Color
c_target = "red"

# Create a 3D-hull scatter plot given covariates X (n x 3) and labels y (n x 1)
def create_figure(X, y, repr, p_x, p_y):

    # Array containing all the clusters and their convex hulls
    layers = []

    unique_labels = np.unique(y)

    k = len(unique_labels)

    color_palette = plt.get_cmap("Greys")(np.linspace(0.2, 0.8, k))  # Using colormap from Matplotlib
    colors = [f'rgba({int(c[0] * 255)}, {int(c[1] * 255)}, {int(c[2] * 255)}, {c[3]})' for c in color_palette]
    
    # Do not plot the marker on initialization
    if p_x is not None:
        user_point = go.Scatter3d(
                    x=[p_x[0]], y=[p_x[1]], z=[p_x[2]], 
                    mode='markers', 
                    marker=dict(
                        size=10,
                        color=c_target,
                        symbol='cross'),
                    name=f'Label {repr[p_y]}',
                    legendgroup=f'Label {repr[p_y]}',
                    showlegend=False,
                )
        layers.append(user_point)

    # Create scatter plots and convex hull volumes for each label in y
    for label in unique_labels:

        # greyscale or label color
        if label == p_y:
            c_val = c_target
        else:
            c_val = colors[label]

        # select rows with corresponding label
        X_k = X[y==label]

        # Compute the convex hull for this group if it has enough points
        if True:
            # hull = ConvexHull(X_k)

            x_c, y_c, z_c  = X_k[:,0], X_k[:,1], X_k[:,2] 

            # Scatter plot for points
            scatter = go.Scatter3d(
                x=x_c, y=y_c, z=z_c, 
                mode='markers', 
                marker=dict(size=4, color=c_val, opacity=1),
                name=f'Label {label}',
                legendgroup=f'Label {label}',
                showlegend=False,
            )
            layers.append(scatter)

            # # Mesh3d for convex hull
            # mesh = go.Mesh3d(
            #     x=x_c,
            #     y=y_c,
            #     z=z_c,
            #     i=hull.simplices[:, 0],
            #     j=hull.simplices[:, 1],
            #     k=hull.simplices[:, 2],
            #     opacity=0.3,
            #     color=colors[label],
            #     hoverinfo='text',
            #     hovertext=f'{repr[label]}',
            #     name=f'Hull {label}',
            #     legendgroup=f'Hull {label}'
            # )
            # layers.append(mesh)

    # Create figure
    fig = go.Figure(data=layers)
    fig.update_layout(plot_bgcolor='lightblue',
                      paper_bgcolor='lightgrey',
                      scene=dict(aspectmode='data',
                                xaxis=dict(visible=False),
                                yaxis=dict(visible=False),
                                zaxis=dict(visible=False)),
                      title="Job Space",
                    )

    return fig

# Layout of the app
app.layout = html.Div([
    form,
    html.Button('Submit', id='submit-button', n_clicks=0),
    dcc.Graph(id='3d-scatter', figure=create_figure(X, labels, representatives, None, None)),
])

@app.callback(
    Output('3d-scatter', 'figure'),
    Input('submit-button', 'n_clicks'),
    # capture the input key-value pairs
    State({'type': 'input-field', 'key': ALL}, 'value'),
    State({'type': 'input-field', 'key': ALL}, 'id'),
    State('3d-scatter', 'figure'),
    prevent_initial_call=True,
)
def update_output(n_clicks, input_values, input_ids, prev_fig):
    if n_clicks > 0:
        if None not in input_values:
            # Create a dictionary to hold form data based on the pattern-matched input fields
            form_data = {input_id['key']: value for input_id, value in zip(input_ids, input_values)}

            # Convert to JSON format
            json_output = json.dumps(form_data, indent=4)
            
            # generate template using json_output
            template = fill_template(input_dict=json_output)
            template_embedding = model.embedder.encode([template])
            X, user_x = model.embedder.get_corrdinates(template_embedding) 
            label = model.predict(template_embedding)
            user_y = model.embedder.rol_to_id_dict.get(label, None)
            print(f"label is: {label} and corresponding encoding is: {user_y}")
            # use template to generate embeddings (user_x) and predict label (user_y)
            # also generate X with reduced dimensions given the user input

            fig = create_figure(X, labels, representatives, user_x, user_y)
        else:
            fig = prev_fig

        return fig



# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8080)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]/Users/rayenebech/Desktop/rayene/phd_application/Helsinki/studies/Period_1/Introduction_to_data_science/proje-env/lib/python3.12/site-packages/transformers/models/mpnet/modeling_mpnet.py:1051: UserWarning:

cumsum_out_mps supported by MPS on MacOS 13+, please upgrade (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/UnaryOps.mm:452.)

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.26it/s]


Dimensionality Reduction with mde took: 19.96953296661377
label is: Sales Operations and corresponding encoding is: 21


Batches: 100%|██████████| 1/1 [00:00<00:00,  7.98it/s]


Dimensionality Reduction with mde took: 19.840790033340454
label is: Administrative-Intern and corresponding encoding is: 0
